### join air quality and grid weather data

In [1]:
import pandas as pd
import numpy as np

In [27]:
# air quality data path
aq_data_path = "../MSBD5002PROJECT_data/air_quality_concated.csv"
gw_data_path = "../MSBD5002PROJECT_data/grid_weather_1701_1804.csv"
geo_info_path = "../MSBD5002PROJECT_data/geo_info_new.csv"

In [41]:
aq_df = pd.read_csv(aq_data_path,index_col=None)
gw_df = pd.read_csv(gw_data_path,index_col=None)
geo_info = pd.read_csv(geo_info_path,index_col=None)

In [29]:
aq_df.info()
gw_df.info()
geo_info.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406910 entries, 0 to 406909
Data columns (total 8 columns):
stationId    406910 non-null object
time         406910 non-null object
PM2.5        353196 non-null float64
PM10         277600 non-null float64
NO2          355093 non-null float64
CO           330655 non-null float64
O3           352928 non-null float64
SO2          355163 non-null float64
dtypes: float64(6), object(2)
memory usage: 24.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7503426 entries, 0 to 7503425
Data columns (total 7 columns):
humidity          float64
pressure          float64
stationId         object
temperature       float64
time              object
wind_direction    float64
wind_speed        float64
dtypes: float64(5), object(2)
memory usage: 400.7+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35 entries, 0 to 34
Data columns (total 11 columns):
aq_name         35 non-null object
longitude       35 non-null float64
latitude        35 non-

In [45]:
def join_aq_gw(aq,gw,geo_info,start_time,end_time):
    aq = aq[(aq.time >= start_time) & (aq.time <= end_time)]
    gw = gw[(gw.time >= start_time) & (gw.time <= end_time)]
    
    # find unique gw_name in geo_info dataframe
    near_gw_stations = geo_info.nearest_gw.unique()
    # filter these gw_station from gw_df
    near_gw_df = gw[gw.stationId.isin(near_gw_stations)]
    
    # first join air quality data and geo_info to add aq_station's nearest gw_station
    geo_info = geo_info.rename(columns={"aq_name":"stationId"})
    aq_gw_foreign = geo_info[["stationId","nearest_gw"]]
    intermediate = aq.join(aq_gw_foreign.set_index("stationId"),on="stationId")
    
    nearest_gw_df = gw.rename(columns={"stationId":"nearest_gw"})
    
    # then join intermediate with grid_data
    joined = intermediate.join(nearest_gw_df.set_index(["nearest_gw","time"]), on=["nearest_gw","time"])
    
    return joined

In [50]:
joined = join_aq_gw(aq_df, gw_df, geo_info,"2017-01-01 00:00:00","2018-04-30 23:00:00")

In [51]:
joined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 406910 entries, 0 to 406909
Data columns (total 14 columns):
stationId         406910 non-null object
time              406910 non-null object
PM2.5             353196 non-null float64
PM10              277600 non-null float64
NO2               355093 non-null float64
CO                330655 non-null float64
O3                352928 non-null float64
SO2               355163 non-null float64
nearest_gw        406910 non-null object
humidity          402920 non-null float64
pressure          402920 non-null float64
temperature       402920 non-null float64
wind_direction    402920 non-null float64
wind_speed        402920 non-null float64
dtypes: float64(11), object(3)
memory usage: 46.6+ MB


In [59]:
# joined[(joined["time"]>="2017-09-30 20:00:00") & (joined["stationId"] == "tiantan_aq")]
# joined[joined["O3"]>=500]
joined.iloc[65824:65840]

,stationId,time,PM2.5,PM10,NO2,CO,O3,SO2,nearest_gw,humidity,pressure,temperature,wind_direction,wind_speed
65824,donggaocun_aq,2017-11-18 04:00:00,9.0,12.0,14.0,0.4,NaN,4.0,beijing_grid_452,13.08,1014.94,0.60,275.72,15.63
65825,donggaocun_aq,2017-11-18 05:00:00,13.0,15.0,17.0,0.4,NaN,4.0,beijing_grid_452,13.21,1013.69,1.57,268.85,15.71
65826,donggaocun_aq,2017-11-18 06:00:00,9.0,19.0,15.0,0.4,NaN,3.0,beijing_grid_452,13.33,1012.45,2.54,262.16,16.02
65827,donggaocun_aq,2017-11-18 07:00:00,12.0,27.0,NaN,0.4,NaN,3.0,beijing_grid_452,14.72,1011.76,1.47,249.82,13.31
65828,donggaocun_aq,2017-11-18 08:00:00,20.0,35.0,NaN,0.5,NaN,4.0,beijing_grid_452,16.10,1011.07,0.41,232.49,11.49
65829,donggaocun_aq,2017-11-18 09:00:00,42.0,61.0,27.0,0.5,NaN,5.0,beijing_grid_452,17.48,1010.38,-0.65,211.41,11.02
65830,donggaocun_aq,2017-11-18 10:00:00,72.0,97.0,34.0,0.6,NaN,10.0,beijing_grid_452,17.73,1010.54,-1.30,216.36,8.09
65831,donggaocun_aq,2017-11-18 11:00:00,113.0,133.0,41.0,0.7,NaN,13.0,beijing_grid_452,17.97,1010.70,-1.95,226.74,5.28
65832,donggaocun_aq,2017-11-18 12:00:00,100.0,130.0,38.0,0.7,NaN,9.0,beijing_grid_452,18.21,1010.85,-2.61,255.90,2.99
65833,donggaocun_aq,2017-11-18 13:00:00,79.0,98.0,41.0,0.7,NaN,8.0,beijing_grid_452,18.30,1011.05,-2.85,277.85,1.72
